In [1]:
!git lfs install
!git clone https://huggingface.co/AlexWortega/instruct_rugptlarge

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'instruct_rugptlarge'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 89 (delta 42), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (89/89), 1.43 MiB | 3.29 MiB/s, done.
Filtering content: 100% (2/2), 4.43 GiB | 86.11 MiB/s, done.


In [3]:
!pip install dask

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.ngc.nvidia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
     |████████████████████████████████| 1.2 MB 9.5 MB/s eta 0:00:01
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.ngc.nvidia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.ngc.nvidia.com'. Adding certificate verification is st

In [1]:
import torch
import re
from multiprocessing import Pool
import wandb
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
from datasets import load_dataset
from torch.utils.data import Dataset
from tqdm import tqdm
from transformers import  AdamW, get_linear_schedule_with_warmup
from torch import optim

In [2]:
!nvidia-smi

Fri May 12 21:51:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   30C    P0    55W / 300W |  25423MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
SEED = 27

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [7]:
conf = OmegaConf.load("config.yaml") #get config

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = GPT2TokenizerFast.from_pretrained(conf.model.model_name_or_path)

In [9]:
special_tokens_dict = {'additional_special_tokens': ['user:', 'bot:']}
tokenizer.add_special_tokens({'eos_token': '<instructionE>'})
tokenizer.add_special_tokens({'bos_token': '<instructionE>'})
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.padding_side = 'right'
tokenizer.add_special_tokens(special_tokens_dict)
model = GPT2LMHeadModel.from_pretrained(conf.model.model_name_or_path)
model.to(device)
model.resize_token_embeddings(len(tokenizer))

Embedding(50266, 1536)

In [10]:
device

device(type='cuda')

In [11]:
dataset_helper = load_dataset(conf.args.dataset_name, cache_dir='.')

No config specified, defaulting to: ru_turbo_saiga/default
Found cached dataset ru_turbo_saiga (/mnt/opt/alexw/Kar/chatbot_saiga_dataset/./IlyaGusev___ru_turbo_saiga/default/0.0.1/28415c7ce4ae7d7de9064f6f7e18a089ba79edce3adc4a43fdbb09e9cd39f16c)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
dataset_chat = load_dataset("zjkarina/matreshka", cache_dir='.')

Using custom data configuration zjkarina--matreshka-8e0974f9d53dcb5f
Found cached dataset parquet (/mnt/opt/alexw/Kar/chatbot_saiga_dataset/./zjkarina___parquet/zjkarina--matreshka-8e0974f9d53dcb5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset_chat

DatasetDict({
    validation: Dataset({
        features: ['role', 'dialog'],
        num_rows: 1736
    })
    train: Dataset({
        features: ['role', 'dialog'],
        num_rows: 6943
    })
})

In [10]:
tokenizer.pad_token_id

50263

In [11]:
tokenizer.eos_token_id

50261

In [27]:
for idx, sen in enumerate(zip(dataset_chat['train'][2]['role'], dataset_chat['train'][2]['dialog'])):
    if sen[0] == 'user' and idx != 0:
        sen = (f" <instructionE> user", sen[1])
        dialog += ': '.join(sen)
    elif sen[0] == 'user' and idx == 0:
        sen = (f"user", sen[1])
        dialog += ': '.join(sen)
    else:
        sen = (f" bot", sen[1])
        dialog += ': <instructionE> '.join(sen)

In [24]:
dataset_saiga = dataset_chat['train'][:5]
for dia in tqdm(dataset_saiga):
    dialog = ""
    for idx, sen in enumerate(zip(dia['role'], dia['dialog'])):
        if sen[0] == 'user' and idx != 0:
            sen = (f" <instructionE> user", sen[1])
            dialog += ': '.join(sen)
        elif sen[0] == 'user' and idx == 0:
            sen = (f"user", sen[1])
            dialog += ': '.join(sen)
        else:
            sen = (f" bot", sen[1])
            dialog += ': <instructionE> '.join(sen)

  0%|          | 0/4 [00:00<?, ?it/s]


TypeError: string indices must be integers

In [28]:
dialog

'user: Как твое настроение сегодня? bot: <instructionE> Привет! Как дела? bot: <instructionE> Мое настроение сегодня нормальное, а у тебя? <instructionE> user: Я думаю, это будет хороший день. А как твои дела?user: Привет! Как дела? bot: <instructionE> Привет! У меня все хорошо, а у тебя? <instructionE> user: У меня тоже хорошо. Знаешь ли ты, что самая высокая гора в мире - это Эверест? bot: <instructionE> Да, я знаю. А ты знаешь, что пингвины могут плавать со скоростью до 22 миль в час? <instructionE> user: Нет, не знал. Интересный факт. А знаешь ли ты, что некоторые киты могут жить до 200 лет? bot: <instructionE> Да, я знаю. Это действительно удивительно. А ты знаешь, что наша галактика Млечный Путь содержит более 200 миллиардов звезд? <instructionE> user: Да, я знаю об этом. Интересно, на сколько еще фактов ты способен? bot: <instructionE> Меня программировали умными людьми, поэтому я знаю много фактов. Например, пчела может облететь до 90 тысяч цветов за один день. <instructionE> u

In [29]:
class DialogDataset(Dataset):
    def __init__(self, tokenizer, dataset_helper, dataset_chat):
        self.tokenized = []
        dataset_saiga = dataset_helper['train']['messages']
        dataset_my_train = dataset_chat['train']
        dataset_my_val = dataset_chat['validation']
        
        for dia in tqdm(dataset_saiga):
            dialog = ""
            for idx, sen in enumerate(zip(dia['role'], dia['content'])):
                if sen[0] == 'user' and idx != 0:
                    sen = (f" <instructionE> user", sen[1])
                    dialog += ': '.join(sen)
                elif sen[0] == 'user' and idx == 0:
                    sen = (f"user", sen[1])
                    dialog += ': '.join(sen)
                else:
                    sen = (f" bot", sen[1])
                    dialog += ': <instructionE> '.join(sen)
            enc = self._encode(text=dialog, tokenizer=tokenizer)
            self.tokenized += [enc]
        
        
        # for input_tokens, output_tokens in samples:
        #     sen = f"{tokenizer.bos_token} bot: {input_tokens} user: {output_tokens} {tokenizer.eos_token}"
        #     enc = self._encode(text=sen, tokenizer=tokenizer)
        #     self.tokenized += [enc]
        for dia in tqdm(dataset_my_train):
            dialog = ""
            try:
                for idx, sen in enumerate(zip(dia['role'], dia['dialog'])):
                    if sen[0] == 'user' and idx != 0:
                        sen = (f" <instructionE> user", sen[1])
                        dialog += ': '.join(sen)
                    elif sen[0] == 'user' and idx == 0:
                        sen = (f"user", sen[1])
                        dialog += ': '.join(sen)
                    else:
                        sen = (f" bot", sen[1])
                        dialog += ': <instructionE> '.join(sen)
                enc = self._encode(text=dialog, tokenizer=tokenizer)
                self.tokenized += [enc]
            except:
                pass
        for dia in tqdm(dataset_my_val):
            dialog = ""
            try:
                for idx, sen in enumerate(zip(dia['role'], dia['dialog'])):
                    if sen[0] == 'user' and idx != 0:
                        sen = (f" <instructionE> user", sen[1])
                        dialog += ': '.join(sen)
                    elif sen[0] == 'user' and idx == 0:
                        sen = (f"user", sen[1])
                        dialog += ': '.join(sen)
                    else:
                        sen = (f" bot", sen[1])
                        dialog += ': <instructionE> '.join(sen)
                enc = self._encode(text=dialog, tokenizer=tokenizer)
                self.tokenized += [enc]
            except:
                pass

    def __len__(self):
        return len(self.tokenized)

    def __getitem__(self, item):
        return self.tokenized[item] 
    
#     @staticmethod
#     def regular(string, tokenizer):
#         new_string = ""
#         slicer = 0
#         counter = 0
#         for match in re.finditer(r'\b\w+:\s', string):
#             new_string += string[slicer:match.start()]# Append the text before the match
#             counter += 1
#             if counter % 2 == 1 and ':' not in new_string[-3:]: # Check if the match is in an even-numbered position
#                 new_string += f" {tokenizer.bos_token} user: "
#             elif counter % 2 == 0 and ':' not in new_string[-3:]:
#                 new_string += " bot: "
#             slicer = match.end() # Update the count to the end of the match

#         new_string += string[slicer:] # Append any remaining text after the last match
#         new_string = re.sub(r'[\r\n]+', '', new_string) # Remove any remaining line breaks
#         return new_string

    @staticmethod
    def _encode(text, tokenizer):
        encoded_sample = tokenizer(text, padding='max_length', max_length=512, truncation=True, return_tensors='pt')

        return encoded_sample

In [30]:
data = DialogDataset(tokenizer=tokenizer, dataset_helper=dataset_helper, dataset_chat=dataset_chat)
train_dataloader = DataLoader(
        data, batch_size=conf.args.batch_size, shuffle=True)

100%|██████████| 1664/1664 [00:01<00:00, 1325.80it/s]


In [31]:
wandb.init(project='diplom', name='frombase_5epoch_saiga+my')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: karina_romanova. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
import torch.nn as nn
class EMA(nn.Module):
    def __init__(self, decay):
        super(EMA, self).__init__()
        self.decay = decay
        self.shadow_params = {}

    def forward(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                if name not in self.shadow_params:
                    self.shadow_params[name] = param.data.clone()
                else:
                    self.shadow_params[name] -= (1 - self.decay) * (self.shadow_params[name] - param.data)
                param.data = self.shadow_params[name]
                
ema = EMA(decay=0.992)

In [19]:
from torch.optim import Optimizer
from typing import Callable, Iterable, Tuple
from torch.distributions.bernoulli import Bernoulli
import math

class ChildTuningAdamW(Optimizer):
    def __init__(
        self,
        params: Iterable[torch.nn.parameter.Parameter],
        lr: float = 1e-3,
        betas: Tuple[float, float] = (0.9, 0.999),
        eps: float = 1e-6,
        weight_decay: float = 0.0,
        correct_bias: bool = True,
        reserve_p = 1.0,
        mode = None
    ):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {} - should be >= 0.0".format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[1]))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {} - should be >= 0.0".format(eps))
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, correct_bias=correct_bias)
        super().__init__(params, defaults)

        self.gradient_mask = None
        self.reserve_p = reserve_p
        self.mode = mode

    def set_gradient_mask(self, gradient_mask):
        self.gradient_mask = gradient_mask

    def step(self, closure: Callable = None):
        """
        Performs a single optimization step.
        Arguments:
            closure (:obj:`Callable`, `optional`): A closure that reevaluates the model and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError("Adam does not support sparse gradients, please consider SparseAdam instead")

                # =================== HACK BEGIN =======================         
                if self.mode is not None:
                    if self.mode == 'ChildTuning-D':
                        if p in self.gradient_mask:
                            grad *= self.gradient_mask[p]
                    else: 
                        # ChildTuning-F
                        grad_mask = Bernoulli(grad.new_full(size=grad.size(), fill_value=self.reserve_p))
                        grad *= grad_mask.sample() / self.reserve_p
                # =================== HACK END =======================

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state["step"] = 0
                    # Exponential moving average of gradient values
                    state["exp_avg"] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state["exp_avg_sq"] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state["exp_avg"], state["exp_avg_sq"]
                beta1, beta2 = group["betas"]

                state["step"] += 1

                # Decay the first and second moment running average coefficient
                # In-place operations to update the averages at the same time
                exp_avg.mul_(beta1).add_(grad, alpha=1.0 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(grad, grad, value=1.0 - beta2)
                denom = exp_avg_sq.sqrt().add_(group["eps"])

                step_size = group["lr"]
                if group["correct_bias"]:  # No bias correction for Bert
                    bias_correction1 = 1.0 - beta1 ** state["step"]
                    bias_correction2 = 1.0 - beta2 ** state["step"]
                    step_size = step_size * math.sqrt(bias_correction2) / bias_correction1

                p.data.addcdiv_(exp_avg, denom, value=-step_size)

                # Just adding the square of the weights to the loss function is *not*
                # the correct way of using L2 regularization/weight decay with Adam,
                # since that will interact with the m and v parameters in strange ways.
                #
                # Instead we want to decay the weights in a manner that doesn't interact
                # with the m/v parameters. This is equivalent to adding the square
                # of the weights to the loss with plain (non-momentum) SGD.
                # Add weight decay at the end (fixed version)
                p.data.add_(p.data, alpha=-group["lr"] * group["weight_decay"])

        return loss

In [ ]:
from transformers import  AdamW, get_linear_schedule_with_warmup
import torch_optimizer

ema = EMA(decay=0.992)
optimizer = AdamW(model.parameters(), lr=1e-3, betas=(0.95, 0.99), weight_decay=0.1)
scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=100, num_training_steps=len(train_dataloader)
            )
model.train()
for epoch in range(5):
    for batch in tqdm(train_dataloader):
    
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        
        # Переносим тензоры на устройство (GPU)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = input_ids

    
        optimizer.zero_grad()
        loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)[0]
        wandb.log({"loss":  loss})

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        loss.backward()
        
        optimizer.step()
        ema(model)
        scheduler.step()
    model.save_pretrained(f'boseos_saiga+my_{epoch+1}')
model.eval()
del optimizer
torch.cuda.empty_cache()

/home/alexw/.local/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 30%|██▉       | 4442/14890 [58:35<2:18:07,  1.26it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 84%|████████▍ | 12565/14890 [2:45:45<30:38,  1.26it/s]  IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_l

In [62]:
 wandb.log({"lossq":  loss})

In [20]:
del optimizer
torch.cuda.empty_cache()

In [21]:
import gc
gc.collect()

270

In [18]:
model.save_pretrained('checkpoint_alldia_in')